In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from pathlib import Path


C:\Users\sys_stephano.antunes\.conda\envs\rag-hospital\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PROJECT_ROOT = Path.cwd().parent
VECTOR_DB_PATH = PROJECT_ROOT / "faiss_index"

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

vectorstore = FAISS.load_local(
    str(VECTOR_DB_PATH),
    embeddings,
    allow_dangerous_deserialization=True
)

print("Vector store carregado com sucesso!")


Vector store carregado com sucesso!


In [3]:
prompt_template = """
Você é um assistente hospitalar para apoio informacional a profissionais de saúde.

Use APENAS as informações fornecidas no contexto abaixo para responder à pergunta.
Se a resposta não estiver claramente nos documentos, diga que não encontrou a informação.

⚠️ IMPORTANTE:
- Não faça diagnósticos médicos
- Não ofereça recomendações clínicas personalizadas
- Não substitua protocolos oficiais da instituição

Contexto:
{context}

Pergunta:
{question}

Resposta (em português, clara e objetiva):
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)


In [4]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


In [5]:
def consultar_chatbot(pergunta, k=3):
    docs = vectorstore.similarity_search(pergunta, k=k)

    context = "\n\n".join([doc.page_content for doc in docs])

    prompt_final = prompt.format(
        context=context,
        question=pergunta
    )

    resposta = llm.invoke(prompt_final)

    return resposta.content


In [8]:
pergunta = "Quais são as principais medidas de biossegurança em ambiente hospitalar?"

resposta = consultar_chatbot(pergunta)

print(resposta)

As principais medidas de biossegurança em ambiente hospitalar incluem:

1. **Manuseio seguro de materiais biológicos**: Utilização de Equipamentos de Proteção Individual (EPIs) adequados, como luvas, máscaras, aventais e protetores faciais, sempre que houver risco de contato com materiais infecciosos.

2. **Descarte adequado de resíduos hospitalares**: Seguir as diretrizes do Ministério da Saúde para o descarte seguro de resíduos, especialmente os que são considerados perigosos.

3. **Avaliação contínua de riscos**: Implementação de uma abordagem multifacetada para a gestão de riscos, com análise dinâmica dos perigos potenciais e ajustes conforme necessário.

4. **Auditorias internas e externas**: Realização de auditorias para identificar falhas e promover melhorias no ambiente hospitalar.

5. **Integração de práticas de biossegurança com gerenciamento de riscos**: Aplicação de métodos sistemáticos para identificação e avaliação de riscos, considerando a probabilidade e o impacto de ev